Intraclass Correlation Coefficient Power Analysis

In [34]:
import numpy as np
import pingouin as pg
import pandas as pd

# def simulate_data(n, mean, std_dev, subject_effects=None):
#     """
#     Simulate data for a single group with shared subject effects.
    
#     n - Number of subjects (sample size).
#     mean - Mean score for the group.
#     std_dev - Standard deviation of the scores for the group.
#     subject_effects - Shared subject effects across groups.
#     """
    
#     # Simulate scores for the group
#     scores = np.random.normal(mean, std_dev, n)
    
#     # Add subject-specific random effects if provided
#     if subject_effects is not None:
#         scores += subject_effects
    
#     return scores


def simulate_data(n, mean_control, std_dev_control, mean_experimental_error, std_dev_experimental_error):
    """
    Simulate data for control and experimental groups with paired sampling.
    
    n - Number of subjects (sample size).
    mean_control - Mean score for the control group.
    std_dev_control - Standard deviation of the scores for the control group.
    mean_experimental - Mean score for the experimental group.
    std_dev_experimental - Standard deviation of the scores for the experimental group.
    subject_effects - Individual subject effects.
    """
    
    # Generate subject effects
    subject_effects = np.random.normal(mean_experimental_error, std_dev_experimental_error, n)
    
    # Generate scores for control group without any adjustment
    control_scores = np.random.normal(mean_control, std_dev_control, n)
    
    # Add subject effects to the experimental scores
    experimental_scores = control_scores + subject_effects
    
    return control_scores, experimental_scores


def compute_icc(control_scores, experimental_scores):
    """
    Compute ICC using the pingouin package.
    
    control_scores - Scores from the control group.
    experimental_scores - Scores from the experimental group.
    """
    # Combine the scores into a single DataFrame
    data = {'Scores': np.concatenate((control_scores, experimental_scores)),
            'Groups': ['Control'] * len(control_scores) + ['Experimental'] * len(experimental_scores),
            'Subjects': list(range(len(control_scores))) * 2}
    df = pd.DataFrame(data)
    
    # Calculate the ICC
    icc = pg.intraclass_corr(data=df, targets='Subjects', raters='Groups', ratings='Scores').set_index('Type').loc['ICC1', 'ICC']
    
    return icc

def power_analysis(target_icc, mean_control, std_dev_control, mean_experimental_error, std_dev_experimental_error, alpha=0.05, power=0.80, min_n=5, max_n=1000, n_sims=100):
    """
    Perform power analysis to find the number of subjects needed to detect a given ICC.
    
    target_icc - The ICC you want to detect.
    mean_control - Mean score for the control group.
    std_dev_control - Standard deviation of the scores for the control group.
    mean_experimental - Mean score for the experimental group.
    std_dev_experimental - Standard deviation of the scores for the experimental group.
    alpha - Significance level, the probability of a Type I error.
    power - The desired power level, probability of correctly rejecting the null hypothesis.
    max_n - Minimum sample size to try.
    max_n - Maximum sample size to try.
    n_sims - Number of simulations for each sample size.
    """
    
    # Loop through sample sizes
    for n in range(min_n, max_n + 1):
        
        # Counter for the number of times the null hypothesis is rejected
        null_rejected = 0
        
        # Shared subject effects
        
        
        # Simulate data and compute ICC for each simulation
        for _ in range(n_sims):
            
            control_scores, experimental_scores = simulate_data(n, mean_control, std_dev_control, mean_experimental_error, std_dev_experimental_error)
            
            # Simulate data
            # control_scores = simulate_data(n, mean_control, std_dev_control, subject_effects)
            # experimental_scores = simulate_data(n, mean_experimental, std_dev_experimental, subject_effects)
            computed_icc = compute_icc(control_scores, experimental_scores)
            
            # Check if the computed ICC meets or exceeds the desired ICC
            if computed_icc >= target_icc:
                null_rejected += 1
        
        # Calculate the achieved power
        achieved_power = null_rejected / n_sims
        
        # Print diagnostic information
        print(f"Sample size: {n}, Achieved Power: {achieved_power}")
        
        # Check if the desired power is achieved
        if achieved_power >= power:
            return n
    
    # Return None if the desired power is not achieved within max_n
    return None


# Example usage:
# required_n = power_analysis(target_icc=0.75, mean_control=90, std_dev_control=3, mean_experimental=90, std_dev_experimental=3, min_n=50, max_n=100, n_sims=100)
# print(f"Required sample size for desired power: {required_n}")


In [60]:
required_n = power_analysis(
    target_icc=0.75,              # Set target ICC to 0.5
    mean_control=60,             # Mean score for the control group
    std_dev_control=5,          # Standard deviation of the scores for the control group
    mean_experimental_error=0,        # Mean score for the experimental group
    std_dev_experimental_error=5,     # Standard deviation of the scores for the experimental group
    min_n=90, max_n=100, n_sims=100)  # Try sample sizes from 5 to 100

print(f"Required sample size for desired power: {required_n}")


Sample size: 90, Achieved Power: 0.01
Sample size: 91, Achieved Power: 0.02
Sample size: 92, Achieved Power: 0.06
Sample size: 93, Achieved Power: 0.04
Sample size: 94, Achieved Power: 0.04
Sample size: 95, Achieved Power: 0.05
Sample size: 96, Achieved Power: 0.04
Sample size: 97, Achieved Power: 0.04
Sample size: 98, Achieved Power: 0.01
Sample size: 99, Achieved Power: 0.02
Sample size: 100, Achieved Power: 0.05
Required sample size for desired power: None
